In [1]:
import pandas as pd
import numpy as np

In [53]:
data = pd.read_csv("woocommerce-ornek-urun-turkce.csv")

In [72]:
data.columns

Index(['Kimlik', 'Tür', 'Stok kodu (SKU)', 'İsim', 'Yayımlanmış', 'Öne çıkan?',
       'Katalogda görünürlük', 'Kısa açıklama', 'Açıklama',
       'İndirimli fiyatın başladığı tarih', 'İndirimli fiyatın bittiği tarih',
       'Vergi durumu', 'Vergi sınıfı', 'Stokta?', 'Stok', 'Düşük stok miktarı',
       'Yok satmaya izin?', 'Ayrı ayrı mı satılıyor?', 'Ağırlık (kg)',
       'Uzunluk (cm)', 'Genişlik (cm)', 'Yükseklik (cm)',
       'Müşteri yorumlarına izin?', 'Satın alma notu',
       'İndirimli satış fiyatı', 'Normal fiyat', 'Kategoriler', 'Etiketler',
       'Gönderim sınıfı', 'Görseller', 'İndirme sınırı',
       'İndirme sona erme günü', 'Ebeveyn', 'Gruplanmış ürünler',
       'Yukarı satışlar', 'Çapraz satışlar', 'Harici URL', 'Düğme metni',
       'Konum', 'İndirme 1 ismi', 'İndirme 1 URL'si', 'Nitelik 1 ismi',
       'Nitelik 1 değer(ler)i', 'Nitelik 1 görünür', 'Nitelik 1 genel',
       'Nitelik 1 varsayılan', 'Nitelik 2 ismi', 'Nitelik 2 değer(ler)i',
       'Nitelik 2 görünür

In [54]:
df = pd.DataFrame(data)

def doldur(column):
    if column.isna().all():
        return column
    else:
        mode_value = column.mode().iloc[0]
        return column.fillna(mode_value)

df = df.apply(doldur)
df = df.dropna(axis=0, how='all')

In [55]:
df.isna().sum().sum()

308

In [65]:
def tum_bos_mu(dataframe):
    newdata = dataframe
    for column in dataframe.columns:
        if dataframe[column].isnull().all():
            print(column)
            newdata = newdata.drop(column, axis=1)
    return newdata

In [66]:
df=tum_bos_mu(df)

İndirimli fiyatın başladığı tarih
İndirimli fiyatın bittiği tarih
Vergi sınıfı
Düşük stok miktarı
Ağırlık (kg)
Uzunluk (cm)
Genişlik (cm)
Yükseklik (cm)
Satın alma notu
Etiketler
Gönderim sınıfı


In [67]:
a = df.isna().sum()
for i, value in a.items():
    if value != 0:
        print(i, value)

In [68]:
df.isna().sum().sum()

0

In [69]:
def sayisal_donusum(dataframe):
    for column in dataframe.columns:
        if dataframe[column].dtype == 'object':
            dataframe[column] = dataframe[column].astype('category').cat.codes
        elif dataframe[column].dtype == 'bool':
            dataframe[column] = dataframe[column].astype(int)
    return dataframe

In [70]:
df = sayisal_donusum(df)

In [78]:
from sklearn.model_selection import train_test_split

def train_test_bol(veri_seti, etiket):
    X = veri_seti.drop(columns=[etiket])
    y = veri_seti[etiket]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_bol(df, 'Stok')


In [ ]:
print("Eğitim Özellikleri:")
print(X_train)
print("\nEğitim Etiketleri:")
print(y_train)
print("\nTest Özellikleri:")
print(X_test)
print("\nTest Etiketleri:")
print(y_test)

In [96]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

def xgboost_regresyon(X_train, X_test, y_train, y_test):
    model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 100)

    model.fit(X_train, y_train)

    train_tahminler = model.predict(X_train)

    test_tahminler = model.predict(X_test)

    eğitim_mse = mean_squared_error(y_train, train_tahminler)
    test_mse = mean_squared_error(y_test, test_tahminler)

    print("Eğitim MSE:", eğitim_mse)
    print("Test MSE:", test_mse)

    return model

model = xgboost_regresyon(X_train, X_test, y_train, y_test)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:2nin Katları: object, 2nin Sonuçları: object

In [92]:
import pandas as pd

# Boş bir DataFrame oluştur
df = pd.DataFrame(columns=['2nin Katları', '2nin Sonuçları', 'Çarpım'])

# İşlemleri yap ve DataFrame'e ekle
for i in range(1, 101):
    katlar = 2 * i
    sonuçlar = 2
    çarpım = katlar * sonuçlar
    df = pd.concat([df, pd.DataFrame({'2nin Katları': [katlar], '2nin Sonuçları': [sonuçlar], 'Çarpım': [çarpım]})], ignore_index=True)

print(df)


   2nin Katları 2nin Sonuçları Çarpım
0             2              2      4
1             4              2      8
2             6              2     12
3             8              2     16
4            10              2     20
..          ...            ...    ...
95          192              2    384
96          194              2    388
97          196              2    392
98          198              2    396
99          200              2    400

[100 rows x 3 columns]


In [93]:
X_train, X_test, y_train, y_test = train_test_bol(df, 'Çarpım')

In [94]:

model = xgboost_regresyon(X_train, X_test, y_train, y_test)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:2nin Katları: object, 2nin Sonuçları: object